# Course 3: Localization
## Part 2: Markov Localization in C++
#### By Jonathan L. Moran (jonathan.moran107@gmail.com)
From the Self-Driving Car Engineer Nanodegree programme offered at Udacity.

## Objectives

* Derive and implement the [Bayes filter](https://en.wikipedia.org/wiki/Recursive_Bayesian_estimation) for localisation;
* Approximate vehicle location as a probability distribution, then perform probabilistic reasoning over time (filtering);
* Use the [Markov Assumption](https://en.wikipedia.org/wiki/Markov_chain) and [law of total probability](https://en.wikipedia.org/wiki/Law_of_total_probability) to initialise the filter with meaningful estimates.

## 1. Introduction

In this part of the Markov Localization lesson we will be programming in C++. In order to follow along with the code in this Jupyter notebook, a C++ kernel will need to be installed. We use the [`xeus-cling`](https://github.com/jupyter-xeus/xeus) kernel, which you can access via [this MyBinder-hosted](https://mybinder.org/v2/gh/QuantStack/xeus-cling/stable?filepath=notebooks/xcpp.ipynb) notebook instance.

## 2. Programming Task

### 2.1. Initialize Priors Function

In this task we will create a function that initialises priors (initial belief state for each position on the map) given landmark positions, a position standard deviation ($\pm 1.0$), and the assumption that our car is parked next to a landmark.

Note that the control standard deviation represents the spread from movement (movement is the result of our control input in this case). We input a control of moving $1.0$ step but our actual movement could be in the range of $\pm 1.0$ control standard deviation. The position standard deviation is the spread in our actual position. For example, we may believe start at a particular location, but we could be anywhere in that location $\pm$ our position standard deviation.

In [1]:
// From J. Moran's `1_initialize_priors.cc`

In [2]:
#include <iostream>
#include <vector>


/* Calculates the 1-D location posterior, i.e., the vector of prior probabilty
 * values for each position in the discretised 1-D pose range space (map) with
 * step resolution of 1.0m. The range space is defined relative to the heading
 * of the ego-vehicle. Here we assume the vehicle starts at one of the
 * `n_landmarks` (static objects) with a position std. deviation of +/- 1.0m.
 */
std::vector<float> initialize_priors(
    int map_size,
    std::vector<float> landmark_positions,
    float position_stdev
) {
  // Initialise the prior probabilities
  std::vector<float> priors(map_size, 0.0);
  // Number of total landmarks in map view
  int n_landmarks = landmark_positions.size();
  // Number of neighbours for each landmark
  int n_neighbours =  2 * int(position_stdev);
  // Number of non-zero priors (landmarks plus neighbours)
  int n_pos = n_landmarks + n_neighbours * n_landmarks;
  // The prior probability of vehicle being at a position of interest
  float p_prior = 1.0 / n_pos;
  // Set the non-zero prior probability values
  for (auto i : landmark_positions) {
      // The landmark prior probability
      priors[int(i)] = p_prior;
      // The neighbouring positions' prior probabilities
      priors[int(i - position_stdev)] = p_prior;  // Left of landmark
      priors[int(i + position_stdev)] = p_prior;  // Right of landmark
  }
  return priors;
}

#### Testing the initialize priors function

For simplicity we assumed a position standard deviation
```cpp
position_stdev = 1.0f
```
and coded a solution for initialising priors accordingly.

Expected result:

```pseudo
0
0
0
0
0.111111
0.111111
0.111111
0
0
0.111111
0.111111
0.111111
0
0
0
0
0
0
0
0.111111
0.111111
0.111111
0
0
0
```

In [3]:
// From J. Moran's `1_tests.cc`

In [4]:
/* Tests the `initialize_priors` function inside `1_initialize_priors`.
 * Returns the initialised 1-D prior probability vector.
 */
void test_initialize_priors() {
  // Set standard deviation of position
  float position_stdev = 1.0f;
  // Set map horizon distance in meters
  int map_size = 25;
  // Initialise landmarks
  std::vector<float> landmark_positions{5, 10, 20};
  // Testing initialise priors function
  std::vector<float> priors = initialize_priors(map_size,
                                                landmark_positions,
                                                position_stdev
  );
  // Print the probability values to `stdout`
  for (int p = 0; p < priors.size(); ++p) {
    std::cout << priors[p] << "\n";
  }
}

In [5]:
test_initialize_priors()

0
0
0
0
0.111111
0.111111
0.111111
0
0
0.111111
0.111111
0.111111
0
0
0
0
0
0
0
0.111111
0.111111
0.111111
0
0
0


Great — our results seem to match the expected! ✅

Note that a more robust approach would be to create a function that accepts a position standard deviation as an argument and adjusts initialisation and normalisation based on its value.

### 2.2. Determine Probabilities

Here we practise using `normpdf` in this workspace, setting a `value` to maximise the probability when given a parameter of `1.0`.

In [6]:
// From J. Moran's `2_determine_probabilities.cc`

In [7]:
#include <cassert>
#include <math.h>

// Define the normalisation term
const static float kOneOverSqrt2Pi = 1.0 / sqrt(2 * M_PI);


class Helpers {
public:
  /* Returns the probability function for value `x` assuming a
   * normal distribution parameterised by (`mu`, `sigma`).
   */
  static float normpdf(float x, float mu, float std) {
    // Distribution parameters must be scalars
    // and std must be positive
    assert(std > 0.0);
    // Compute the probability function
    return (kOneOverSqrt2Pi / std) * exp(-0.5 * pow((x - mu) / std, 2));
  }
};

#### Testing determine probabilities function

Here we select a position
```cpp
value = 1.0
```
which maximises the control parameter when `parameter = 1.0`.

In [8]:
// From J. Moran's `1_tests.cc`

In [9]:
#include <iostream>

/* Tests the `normpdf` function inside `2_determine_probabilities`.
 * Returns the probability value obtained from the normal distribution
 * parameterised with the given scalar values of (`mu`, `sigma`).
 */
void test_determine_probabilities() {
  // The position `x`
  float value = 1.0;
  // The position / observation control parameter
  float parameter = 1.0;    // Number of units moved each time-step
  // The position / observation standard deviation
  float stdev = 1.0;
  // Compute the probability distribution for the given values
  static float prob = Helpers::normpdf(value, parameter, stdev);
  // Print the returned probability value
  std::cout << prob << std::endl;
}

In [10]:
test_determine_probabilities()

0.398942


The resulting probability value will always be maximised when the control `parameter` and position `value` are equal. Generally speaking, we should observe a maximum probability value at:

```cpp
x = value - control
```

Note that we assume here a control parameter of `1.0` to represent the number of moves per unit of time. Therefore, the vehicle is assumed to have moved `1.0` units per time-step.

### 2.3. Motion Model Probability II

In this task we extend the functionality of the `normpdf` function from Sect. 2.2. in order to compute the probability for a given pseudo-position.

Here, we simply modify the `value` parameter to be the pseudo-position delta, i.e., the difference between the pseudo- and pre-pseudo positions. 

Assuming we have an initial pseudo-position of
```cpp
x_pseudo = 7
```
and an initial pre-pseudo position
```cpp
x_pre_pseudo = 5
```
we can therefore compute the `value` parameter as such:
```cpp
value = x_pseudo - x_pre_pseudo;
```
which results in a value of $5$.


Let's see what the probability value is from the `normpdf` function for this position delta:

In [11]:
// From J. Moran's `1_tests.cc`

In [12]:
#include <iostream>

/* Tests the `normpdf` function inside `2_determine_probabilities`.
 * Returns the probability value obtained from the normal distribution
 * parameterised with the given scalar values of (`mu`, `sigma`).
 */
void test_motion_model_probability() {
  // Defining the distribution parameters
  float control_parameter = 1.0f;    // Number of units moved each time-step
  float position_stdev = 1.0f;       // Position / observation standard deviation
  // Computing the position delta
  int x_pseudo = 7;
  int x_pre_pseudo = 5;
  float x_delta = x_pseudo - x_pre_pseudo;
  std::cout << "Position delta: " << x_delta << "\n";
  // Computing the probability value
  static float prob = Helpers::normpdf(x_delta,
                                       control_parameter,
                                       position_stdev
  );
  // Printing the returned probability value
  std::cout << prob << "\n";
}

#### Testing the motion model probability function

In [13]:
test_motion_model_probability()

Position delta: 2
0.241971


### 2.4. Coding the Motion Model

Now that we have manually calculated each step for determining the motion model probability, we will implement these steps in a separate function. The `motion_model` function should perform the following steps for each position in $x$:
1. Loop over all positions in vector `x`;
    * (a) Calculate the transition probability for each prior $x_{t-1}$;
    * (b) Calculate the discrete motion model probability as the product of the transition model probability and the belief state (prior) of $x_{t-1}$;
2. Compute and return the total probability as the sum over each discrete probability value obtained in Step 1.  

In [14]:
// From J. Moran's `4_coding_the_motion_model.cc`

In [15]:
#include <iostream>
#include <vector>

// Forward declaration of `initialize_priors` from `1_initialize_priors.cc`
// std::vector<float> initialize_priors(
//     int map_size,
//     std::vector<float> landmark_positions,
//     float position_stdev
// );

/* Returns the motion model probability value, i.e., the likelihood
 * of being at a given position at the current time-step.
 */
float motion_model(
    float pseudo_position,
    float movement,
    std::vector<float> priors,
    int map_size,
    int control_stdev
) {
  // Initialise the position probability
  float prob_position = 0.0f;
  // 1. Loop over all positions in state space (performing convolution)
  for (int j = 0; j < map_size; j++) {
    float pseudo_position_next = float(j);
    // 1a. Compute the transition probability
    float dist = pseudo_position - pseudo_position_next;
    float prob_transition = Helpers::normpdf(dist,
                                             movement,
                                             control_stdev
    );
    // Estimate the motion model probability with the product rule
    prob_position += prob_transition * priors[j];
  }
  // Return the motion model probability value
  return prob_position;
}

#### Testing the motion model function

In [16]:
// From J. Moran's `1_tests.cc`

In [17]:
#include <iostream>
#include <vector>

void test_motion_model() {
  // Setting the distribution parameters
  float control_stdev = 1.0f;
  float position_stdev = 1.0f;
  // Setting the map parameters
  float movement_per_timestep = 1.0f;  // Number of steps (metres)
  int map_size = 25;                   // Number of discrete positions on map
  // Initialise the landmarks
  std::vector<float> landmark_positions{5, 10, 20};
  // Initialise the prior probability vector
  std::vector<float> priors = initialize_priors(map_size,
                                           landmark_positions,
                                           position_stdev
  );
  // 1. Loop over all pseudo-positions $x_{t}^{(i)}$
  for (int i = 0; i < map_size; i++) {
    float pseudo_position = float(i);
    // 1a-b. Compute the transition and discrete motion probability
    float motion_prob = motion_model(pseudo_position,
                                     movement_per_timestep,
                                     priors,
                                     map_size,
                                     control_stdev
    );
    // Print the resulting motion model probability
    std::cout << pseudo_position << "\t" << motion_prob << "\n";
  }
}

In [18]:
test_motion_model()

0	1.65867e-07
1	1.50359e-05
2	0.000507463
3	0.00650629
4	0.0333771
5	0.0772117
6	0.0981132
7	0.077719
8	0.0398834
9	0.0398834
10	0.077719
11	0.0981132
12	0.0772117
13	0.0333771
14	0.00650629
15	0.000507629
16	3.00718e-05
17	0.000507629
18	0.00650629
19	0.0333771
20	0.0772116
21	0.0980982
22	0.0772116
23	0.0333771
24	0.00650629


With that we have successfully implemented the motion model probability function. Let's finish off the Markov Localization implementation in C++ with a few more helper functions.

### 2.5. Observation Model Probability

In this task we In this task we extend the functionality of the `normpdf` function from Sect. 2.2. in order to compute the observation model probability for a given pseudo-position.

Here, we modify the `value` parameter to be the pseudo-position delta, i.e., the difference between the pseudo- and pre-pseudo positions. We also modify the control `parameter` to be observation measurement or control parameter, and the `stdev` to be the position or observation standard deviation.

Note that here we assume the observation measurement / pseudo-distance pairs have already been formed. In order to understand how this was computed, see Sect. 2.4. of [`2022-11-25-Course-3-Localization-Exercises-Part-1.ipynb`](https://github.com/jonathanloganmoran/ND0013-Self-Driving-Car-Engineer/blob/3.1/3-Localization/3-1-Markov-Localization/2022-11-25-Course-3-Localization-Exercises-Part-1.ipynb).

In [19]:
// From J. Moran's `1_tests.cc`

In [20]:
#include <iostream>

/* Tests the `normpdf` function inside `2_determine_probabilities`.
 * Returns the probability value obtained from the normal distribution
 * parameterised with the given scalar values of (`mu`, `sigma`).
 */
void test_motion_model_probability() {
  // Defining the distribution parameters
  float observation_stdev = 1.0f;
  // The observation measurements
  float observation_measurement_1 = 5.5;
  float observation_measurement_2 = 11.0;
  // The pseudo-range estimates
  float x_delta_1 = 5.0;
  float x_delta_2 = 11.0;
  // Computing the probability of the first pair
  // See: `2022-11-25-Course-3-Localization-Exercises-Part-1.ipynb` 
  std::cout << "Pair 1: [" << observation_measurement_1 << ", " << x_delta_1 << "]\n";
  static float prob1 = Helpers::normpdf(observation_measurement_1,
                                       x_delta_1,
                                       observation_stdev
  );
  std::cout << prob1 << "\n";
  std::cout << "-----------------------" << "\n";
  // Computing the probability of the second pair
  std::cout << "Pair 2: [" << observation_measurement_2 << ", " << x_delta_2 << "]\n";
  static float prob2 = Helpers::normpdf(observation_measurement_2,
                                       x_delta_2,
                                       observation_stdev
  );
  std::cout << prob2 << "\n";
}

#### Testing the observation probability function

In [21]:
test_motion_model_probability()

Pair 1: [5.5, 5]
0.352065
-----------------------
Pair 2: [11, 11]
0.398942


### 2.6. Get Pseudo Ranges

In the previous exercises in [`2022-11-25-Course-3-Localization-Exercises-Part-1.ipynb`](https://github.com/jonathanloganmoran/ND0013-Self-Driving-Car-Engineer/blob/3.1/3-Localization/3-1-Markov-Localization/2022-11-25-Course-3-Localization-Exercises-Part-1.ipynb) we manually executed the steps for determining pseudo-range estimates and the overall observation model probability. Now, let's implement a function that accepts a vector of landmark positions — the pseudo positions `x`, that returns a vector of sorted (ascending) pseudo-range estimates. The output vector of this function will be later used as input to our observation model function.

To implement the `pseudo_range_estimator` function, we must do the following for each pseudo-position $x_{t}^{(i)}$:

1. Loop over all landmark positions:
    * (a) Calculate the distance between each pseudo-position $x_{t}^{(i)}$ and the given landmark;
    * (b) Append the distance to the pseudo-range vector if its value is positive (i.e., the landmark is in front of the pseudo-position);
2. Return the resulting pseudo-range vector, sorted in ascending order.

Note that the resulting pseudo-range vector may not contain all values from the input `x`. That is because we assume not all values of `x` have landmark positions in front of the vehicle's current position. 

In [22]:
// From J. Moran's `6_get_pseudo_ranges.cc`

In [23]:
#include <algorithm>
#include <vector>

/* Returns the pseudo-range estimates for a given set
* of landmark positions. The positions are defined
* in 1-D global map coordinates with respect to the
* heading of the ego-vehicle (i.e., the forward motion).
*/
std::vector<float> pseudo_range_estimator(
  std::vector<float> landmark_positions,
  float pseudo_position
) {
  // Initialise the pseudo-observation vector and distance
  std::vector<float> pseudo_ranges(0.0, landmark_positions.size());
  float dist = 0.0;
  // 1. Loop over landmarks
  for (auto l_pos : landmark_positions) {
    // 1a. Compute the distance between position and landmark
    dist = l_pos - pseudo_position;
    if (dist > 0.0) {
      // Landmark is in front of pseudo-position, append to list
      pseudo_ranges.push_back(dist);
    }
    else {
      // Skip the landmark, could be behind vehicle
      continue;
    }
  }
  // Sort the resulting vector in ascending order
  std::sort(pseudo_ranges.begin(), pseudo_ranges.end());
  return pseudo_ranges;
}

#### Testing the pseudo-ranges function

In [24]:
// From J. Moran's `1_tests.cc`

In [25]:
#include <iostream>
#include <vector>


/* Tests the `pseudo_range_estimator` function inside `3_get_pseudo_ranges.cc`.
 * Returns the pseudo-range vector containing the landmark distances relative
 * to the given `pseudo_position`.
 */
void test_pseudo_range_estimator() {
  // The landmark positions in 1-D map space
  std::vector<float> landmark_positions{5, 10, 12, 20};
  // The number of discrete positions `x` on the map
  int map_size = 25;
  // Number of metres moved by the vehicle per time-step
  float movement_per_timestep = 1.0f;
  // The standard deviation of the control
  float control_stdev = 1.0f;
  // Compute the pseudo-ranges for each position on the map
  for (int i = 0; i < map_size; i++) {
    float pseudo_position = float(i);
    std::vector<float> pseudo_ranges = pseudo_range_estimator(landmark_positions,
                                                              pseudo_position
    );
    // Print the resulting pseudo-range vector values
    if (pseudo_ranges.size()) {
      for (auto val : pseudo_ranges) {
        std::cout << "x: " << i << "\t" << val << "\n";
      }
      std::cout << "-----------------------" << "\n";
    }
    else {
      // No landmarks in front of vehicle
      continue;
    }
  }
}

In [26]:
test_pseudo_range_estimator()

x: 0	5
x: 0	10
x: 0	12
x: 0	20
-----------------------
x: 1	4
x: 1	9
x: 1	11
x: 1	19
-----------------------
x: 2	3
x: 2	8
x: 2	10
x: 2	18
-----------------------
x: 3	2
x: 3	7
x: 3	9
x: 3	17
-----------------------
x: 4	1
x: 4	6
x: 4	8
x: 4	16
-----------------------
x: 5	5
x: 5	7
x: 5	15
-----------------------
x: 6	4
x: 6	6
x: 6	14
-----------------------
x: 7	3
x: 7	5
x: 7	13
-----------------------
x: 8	2
x: 8	4
x: 8	12
-----------------------
x: 9	1
x: 9	3
x: 9	11
-----------------------
x: 10	2
x: 10	10
-----------------------
x: 11	1
x: 11	9
-----------------------
x: 12	8
-----------------------
x: 13	7
-----------------------
x: 14	6
-----------------------
x: 15	5
-----------------------
x: 16	4
-----------------------
x: 17	3
-----------------------
x: 18	2
-----------------------
x: 19	1
-----------------------


Fantastic — Our results match the expected output! 🎉

### 2.7. Coding the Observation Model

In Sect. 2.4 we implemented the motion model successfully. Now, in order to complete the 1-D Markov Localization filter in C++, we must write the function needed to compute the observation model. This `observation_model` function should have as input parameters an observation vector (from the vehicle sensors) and the previously-computed pseudo-range vector from Sect. 2.5. The function should return the observation model probability vector, which will be used to compute the final total probability as a product of the normalised motion and observation model results.

The `observation_model` function should perform the following steps:
1. Loop over each observation in vector $z_{t}$:
    * (a) Calculate landmarks in front of the given pseudo-range positions (if any);
    * (b) If the result of 1(a) is non-zero, extract the minimum distance (i.e., element at index 0 from the sorted `pseudo_ranges` vector);
    * (c) Use the minimum distance from 1(b) to obtain the probability value returned by `normpdf`. If the result of 1(a) was an empty vector (i.e., no landmarks in front of pseudo-postiion), then use a maximum distance value (e.g., `infinity`) as input to `normpdf`;
2. Return the likelihood value computed for all observations in vector $z_{t}$.

In [27]:
// From J. Moran's `7_coding_the_observation_model.cc`

In [28]:
#include <vector>


/* Returns the observation probability, i.e., the likelihood
 * value for a given set of `observations` and `pseudo_ranges`.
 * Here we assume the `pseudo_ranges` vector is sorted in ascending
 * order, such that the minimum landmark distance relative to the vehicle
 * occurs at the first index, i.e., `pseudo_ranges[0]`. If no landmarks
 * exist relative to the vehicle heading (i.e., `psuedo_ranges` is empty),
 * then the distance is initialised to a very large number (i.e., `infinity`).
 */
float observation_model(
    std::vector<float> landmark_positions,
    std::vector<float> observations,
    std::vector<float> pseudo_ranges,
    float distance_max,
    float observation_stdev
) {
  // Initialise the observation likelihood value
  float observation_likelihood = 1.0f;
  // 1. Loop over all observations
  for (auto z_i : observations) {
    // 1b. Extract the minimum distance from sorted `pseudo_ranges`
    float dist_min;
    // 1a. Check if pseudo-range vector is non-empty
    if (!pseudo_ranges.empty()) {
      // Assuming `pseudo_ranges` is sorted in ascending order
      dist_min = pseudo_ranges[0];
      // Remove the minimum distance from the vector
      pseudo_ranges.erase(pseudo_ranges.begin());
    }
    else {
      // 1c. No observations in front of vehicle
      dist_min = std::numeric_limits<const float>::infinity();
    }
    // 1c. Compute the probability using `normpdf`
    float p_obs = Helpers::normpdf(z_i,
                                   dist_min,
                                   observation_stdev
    );
    // Update the observation likelihood for this observation
    // using the product rule (i.e., intersection of events)  
    observation_likelihood *= p_obs;
  }
  return observation_likelihood;
}

#### Testing the observation model function

In [29]:
// From J. Moran's `1_tests.cc`

In [30]:
#include <iostream>
#include <vector>


/* Tests the `observation_model` function in `7_coding_observation_model.cc`.
 * Prints the resulting observation likelihood computed at each pseudo-map
 * position, given the distribution and map parameters and the observation
 * vector, i.e., set of sensor readings obtained from the vehicle.
 */
void test_observation_model() {
  // Define the distribution parameters
  float observation_stdev = 1.0f;
  // Define the map parameters
  int map_size = 25;                  // Number of discrete positions on map
  float dist_max = map_size;          // Maximum position on map
  std::vector<float> landmark_positions{
      5, 10, 15, 20
  };                                  // Ground-truth landmark positions on map
  std::vector<float> observations{
      5.5, 13.0, 15.0
  };                                  // Measurements from vehicle sensor
  // Loop through each pseudo-position on the map
  for (int i = 0; i < map_size; i++) {
    float pseudo_position = float(i);
    // Compute the pseudo-range distances from vehicle to map position
    std::vector<float> pseudo_ranges = pseudo_range_estimator(landmark_positions,
                                                              pseudo_position
    );
    // Compute observation likelihood at this position
    float observation_likelihood = observation_model(landmark_positions,
                                                     observations,
                                                     pseudo_ranges,
                                                     dist_max,
                                                     observation_stdev
    );
    // Print the resulting likelihood value
    std::cout << "Pseudo-position (x): " << pseudo_position << "\n";
    std::cout << observation_likelihood << "\n";
    std::cout << "-----------------------" << "\n";
  }
}    

In [31]:
test_observation_model()

Pseudo-position (x): 0
0.00062247
-----------------------
Pseudo-position (x): 1
4.19417e-06
-----------------------
Pseudo-position (x): 2
1.40699e-09
-----------------------
Pseudo-position (x): 3
2.34991e-14
-----------------------
Pseudo-position (x): 4
1.95402e-20
-----------------------
Pseudo-position (x): 5
0.00062247
-----------------------
Pseudo-position (x): 6
4.19417e-06
-----------------------
Pseudo-position (x): 7
1.40699e-09
-----------------------
Pseudo-position (x): 8
2.34991e-14
-----------------------
Pseudo-position (x): 9
1.95402e-20
-----------------------
Pseudo-position (x): 10
0
-----------------------
Pseudo-position (x): 11
0
-----------------------
Pseudo-position (x): 12
0
-----------------------
Pseudo-position (x): 13
0
-----------------------
Pseudo-position (x): 14
0
-----------------------
Pseudo-position (x): 15
0
-----------------------
Pseudo-position (x): 16
0
-----------------------
Pseudo-position (x): 17
0
-----------------------
Pseudo-posit

### 2.8. Coding the Full Filter

In [ ]:
TODO.

#### Testing the 1-D Markov Localization filter in C++

In [ ]:
// From J. Moran's `8_coding_the_full_filter.cc`

In [ ]:
// From J. Moran's `1_tests.cc`

## Credits

This assignment was prepared by Aaron Brown, Tiffany Huang and Maximilian Muffert of Mercedes-Benz Research & Development of North America (MBRDNA), 2021 (link [here](https://www.udacity.com/course/self-driving-car-engineer-nanodegree--nd0013)).